In [1]:
import numpy as np

In [2]:
data = np.load("pact_dataset_ctx16.npz", allow_pickle=True)

In [3]:
# data = np.load("pact_dataset_ctx16.npz", allow_pickle=True)
data

NpzFile 'pact_dataset_ctx16.npz' with keys: state_ctx, action_ctx, target_action, valid_mask, episode...

In [5]:
data['state_ctx'].shape

(512, 16, 38)

In [7]:
data['action_ctx']

array([[[ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        ...,
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [-7.4633515e-01,  2.6657838e-01]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        ...,
        [ 0.0000000e+00,  0.0000000e+00],
        [-7.4633515e-01,  2.6657838e-01],
        [-6.8383753e-01,  2.4425530e-01]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        ...,
        [-7.4633515e-01,  2.6657838e-01],
        [-6.8383753e-01,  2.4425530e-01],
        [-6.1303717e-01,  1.6914645e-01]],

       ...,

       [[-1.9440800e-03,  1.6478677e-03],
        [-2.8699562e-03,  6.0646719e-04],
        [-3.2943934e-03,  7.4626347e-05],
        ...,
        [-2.9486381e-03, -5.6967093e-04],
     

In [8]:
# model_and_tokenizer.py
import torch
import torch.nn as nn

# adjust the import path to your project layout if needed
from src.models.modules.modeling_pact import PACTBase
# PACTBase wraps PACTTokenizer internally and builds the GPT backbone. :contentReference[oaicite:6]{index=6}

def build_pact_model(state_dim=38, action_dim=2, ctx_tokens=16,
                     n_embd=128, n_layer=4, n_head=8):
    """
    ctx_tokens = number of tokens the transformer sees (must be even).
    Because GPT block_size = 2 * seq_len, set seq_len = ctx_tokens // 2.  :contentReference[oaicite:7]{index=7}
    """
    assert ctx_tokens % 2 == 0, "ctx_tokens must be even"
    seq_len = ctx_tokens // 2

    gpt_config = dict(
        n_embd=n_embd, n_layer=n_layer, n_head=n_head,
        embd_pdrop=0.1, resid_pdrop=0.1, attn_pdrop=0.1,
        seq_len=seq_len
    )

    # Different tokenizers for "state" vs "action"
    # state -> VectorStateTokenizer (mlp_state)
    # action -> ActionTokenizer (mlp_action)
    # The action tokenizer handles both continuous and discrete; we use "continuous" here. :contentReference[oaicite:8]{index=8} :contentReference[oaicite:9]{index=9}
    input_config = {
        "state": {
            "tokenizer": "mlp_state",
            "input_type": "continuous",
            "tokenizer_kwargs": {"state_dim": state_dim, "hidden": [256, 256], "use_ln": True},
        },
        "action": {
            "tokenizer": "mlp_action",
            "input_type": "continuous",  # change to "discrete" if your actions are ids
            "tokenizer_kwargs": {"action_dim": action_dim, "hidden": [128, 128], "use_ln": True},
        },
    }

    model = PACTBase(gpt_config=gpt_config, input_config=input_config)
    return model


ModuleNotFoundError: No module named 'src'

In [10]:
from a import run_demo

# points to your two-episode, 256-step dataset
res = run_demo("pact_dataset.npz",
               state_key="full_obs",
               action_key="taken_action")   # or "nominal_action"


ModuleNotFoundError: No module named 'src'

In [ ]:
res